# Problem set 2
The purpose of this and next week's exercises is to estimate two basic linear panel data models
with unobserved effects. The two models make different assumptions about the correlation
between observed and unobserved components and it it is important to understand which set
of assumptions are the most appropriate in empirical applications. Next week's exercise goes
through an econometric test procedure (the Hausman test) that tests the assumptions of the two
models against each other. This week's exercise starts out by estimating the unobserved effects
model allowing for arbitrary contemporaneous correlation between the unobserved individual
effect and the explanatory variables. We shall use two estimators: The Fixed-Effects (FE)
estimator and the First-Difference (FD) estimator. <br>

Before we start working on some exercises we will briefly introduce two concepts in Python. First, importing and exporting data. Second, using functions. If you are already familiar
with these features, you can skip the next two sections and jump directly to the exercises.

First, import all necessary packages. We have made a .py file that we will use as a "toolbox". We will fill this toolbox with functions, that we will use as we progress through the course. Exactly how you structure this toolbox is up to you (if you i.e. want to turn it into a class).

In [1]:
import numpy as np
from numpy import linalg as la
import pandas as pd
from io import StringIO
from tabulate import tabulate
from matplotlib import pyplot as plt

# Import this weeks LinearModels .py file
import LinearModelsWeek2_ante as lm
%load_ext autoreload
%autoreload 2

## Importing and exporting data in Python
The easiest way to import data into an numpy array is using a .txt file. Normally we specify a path to the text file, but we will create a fake one to illustrate.

In [ ]:
# Create a fake file for easy use.
fake_file = StringIO("0 1\n 2 3")
print(f"Fake file looks like this: \n {fake_file.getvalue()}")
print()

# Load the fake txt file into a numpy array.
data = np.loadtxt(fake_file)
print(f'Loaded into a numpy array, we get the following {type(data)}: \n {data}')

Sadly, there is no direct way to load an excel sheet into numpy. The easiest solution is to use pandas as an intermediate.

In [ ]:
# We save the fake file we created earlier as an excel file, 
# so that we can illustrate how to import using excel.
to_export = pd.DataFrame(data)
to_export.to_excel('test_file.xlsx', header=None, index=None)

# Its important to note that Pandas will treat the first row as a header. If there is no header,
# this needs to be specified. There are also alot of extra options to load specific sheets, or
# only parts of the sheets and tons of extra options.
df_import = pd.read_excel('test_file.xlsx', header=None)
np_array = df_import.to_numpy()
print(np_array)

### Exporting Data
To save a numpy array as a .txt file is easy:

In [ ]:
np.savetxt('real_file.txt', np_array)

*If one have large numpy arrays and want to store them efficiently, they can be saved as a binary .npy files. Such files are not compatible with other programs.*

## Excercise with within-groups estimation (FE)

Consider the following linear model,

$$ y_{it} = \boldsymbol{x}_{it}\boldsymbol{\beta} + c_i + u_{it}, \tag{1} $$

where $i = 0, ..., N$ indexes the cross sectional unit that is observed (e.g., households), and $t = 0, ..., T$ denotes time (e.g. weeks, years). $\boldsymbol{x}_{it}$ is a $1 \times K$ vector of regressors, $\boldsymbol{\beta}$ contains the $K \times 1$ parameters to be estimated, $c_i$ is an unobserved individual specifc component which is constant across time periods, and $u_{it}$ is an unobserved random error term.  <br>
If $c_i$ turns out to be an additional error term correated with the regressors in the sense of $E[c_i\boldsymbol{x}_{it}]=0$ for all $t$, then $\boldsymbol{\beta}$ can be consistently estimated by pooled OLS (POLS) (as $N\rightarrow \infty$ for fixed $T$), albeit in an inefficient manner. To see this, consider the joint error termt $v_{it} = c_i + u_{it}$, and note that,
$$E[v_{it}\boldsymbol{x}_{it}] = E[c_{i}\boldsymbol{x}_{it}] + E[u_{it}\boldsymbol{x}_{it}] = \boldsymbol{0},$$

so that the usual OLS assumptions are satisfied. Conversely, if $c$ is systematically related to one or more of the observed variables in the sense of $E[c_{i}\boldsymbol{x}_{it}] \neq \boldsymbol{0}$, then the POLS estimator is _not_ consistent for $\boldsymbol{\beta}$.

### Example 1. 
When might $E[c_{i}\boldsymbol{x}_{it}] \neq \boldsymbol{0}$? Consider a model designed to investigate if union membership affects wages. The model explains wages as a function of experience and their union membership.

$$ ln(wage_{it}) = \beta_1 exper_{it} + \beta_2 exper^2_{it} + \beta_3 union_{it} + c_i + u_{it}, $$

where $c_i$ is an individual-specific constant that summarizes innate and unobserved characteristics. If people select into union or non-union jobs based on which sector rewards their innate characteristics best, then $E[uniont_{it}c_i]\neq0$. For this reason, it doesn't seem reasonable to use OLS on the pooled data. <br>
In this example, the inconsistency of OLS is caused by the presence of $c$. The conventional approach to deal with this problem in linear panel data models is to transform equation (1) such that $c$ vanishes, and the transformed model allows $\boldsymbol{x}$ to be estimated by OLS. Because the model is linear, we may rid ourselves of $c$ using relatively simple, linear, transformations. In the following, we shall consider two such transformations: i) the _within-groups_ transformation, and ii) the _first-difference_ transformation.

## Fixed Effects and Within-Groups Transformation

The within-groups transformation subtracts from each variable
its mean within each cross sectional unit. Consequently, every time-invariant variables disappear when using this transformation. To make the within-groups transformation more explicit, take the average of equation (1) across $T$ for each $i$ to obtain

\begin{equation}
\bar{y}_{i}=\mathbf{\bar{x}}_{i}\mathbf{\beta}+c_{i}+\bar{u}_{i}, \tag{2}
\end{equation}

where $\bar{y}_{i}=T^{-1}\sum_{t=1}^{T}y_{it}$, $\mathbf{\bar{x}}_{i}=T^{-1}\sum_{t=1}^{T}\mathbf{x}_{it}$, and $T^{-1}\sum_{t=1}^{T}c_{i}=c_{i}$.
Subtract equation (2) from equation (1) to get

\begin{align}
y_{it}-\bar{y}_{i} & =\left(\mathbf{x}_{it}-\mathbf{\bar{x}}_{i}\right)\mathbf{\beta}+(\color{red}{c_{i}-c_{i}})+\left(u_{it}-\bar{u}_{i}\right) \\
\Leftrightarrow\ddot{y}_{it} & =\ddot{\mathbf{x}}_{it}\mathbf{\beta} + \ddot{u}_{it}. \tag{3}
\end{align}

This simple manipulation of the empirical model has eliminated
$c_{i}$ by subtracting the mean within each *i*-group. This
is called the *within transformation*, and a within-transformed
variable is denoted $\ddot{y}_{it}=y_{it}-\bar{y}_{i}$. The parameters of interest, $\boldsymbol{\beta},$ can be estimated by OLS on the transformed data, i.e.

\begin{equation}
\hat{\mathbf{\beta}}_{FE}=(\mathbf{\ddot{X}}'\mathbf{\ddot{X}})^{-1}\mathbf{\ddot{X}}'\ddot{\mathbf{y}},
\end{equation}

where $\mathbf{\ddot{X}}$ is the $NT\times K$ matrix and $\ddot{\mathbf{y}}$ the $NT\times1$ vector arising from stacking the observables of (3), i.e., $\ddot{\mathbf{x}}_{it}'$ and $\ddot{y}_{it}$, over first $t$ and then $i$.

## FE Assumptions

Let $\mathbf{\ddot{X}}_{i}$ denotes the $T\times K$ matrix arising
from stacking $\ddot{\mathbf{x}}_{it}'$ over $t$. (We here keep
the $i$ subscript to avoid a clash of notation.) We invoke the following assumptions

\begin{eqnarray*}
FE.1 & : & E[u_{it}|\mathbf{x}_{i1},..,\mathbf{x}_{iT},c_{i}]=0,\quad\text{ for }t=1,\dotsc,T,\\
FE.2 & : & \text{Rank }E[\mathbf{\ddot{X}}_{i}'\mathbf{\ddot{X}}_{i}]=K,\quad\text{ for }i=1,\dotsc,N\\
FE.3 & : & E[\mathbf{u}_{i}\mathbf{u}_{i}'|\mathbf{x}_{i},c_{i}]=\sigma_{u}^{2}\mathbf{I}_{T},\quad\text{ for }i=1,\dotsc,N.
\end{eqnarray*}

Under the strict exogeneity assumption ($FE.1$) and the rank condition
($FE.2$), the FE estimator, $\hat{\mathbf{\beta}}_{FE}$, consistently estimate $\mathbf{\beta}$ as $N\to\infty$ for fixed $T$. Under FE.3, $\hat{\mathbf{\beta}}_{FE}$ is also asymptotically efficient. (But the latter assumption is not needed for consistency.)

In order to perform inference on the obtained parameter
estimates, we need standard errors of the parameter estimates. If
the unobservables $\{u_{it}\}_{t=1}^{T}$ of (1) satisfy
$FE.3$, then the variance-covariance matrix of the FE estimator may
be estimated by

\begin{equation}
\widehat{\mathrm{var}}(\hat{\mathbf{\beta}}_{FE})=\hat{\sigma}_{u}^{2}(\mathbf{\ddot{X}}'\mathbf{\ddot{X}})^{-1},
\end{equation}

where $\hat{\sigma}_{u}^{2}:=\widehat{\ddot{\mathbf{u}}}'\widehat{\ddot{\mathbf{u}}}/[N\left(T-1\right)-K]$
and $\widehat{\ddot{\mathbf{u}}}:=\ddot{\mathbf{y}}-\mathbf{\ddot{x}}\mathbf{\beta}$ so that $\widehat{\ddot{\mathbf{u}}}'\widehat{\ddot{\mathbf{u}}}=\sum_{i=1}^{N}\sum_{t=1}^{T}\hat{\ddot{u}}_{it}^{2}$.


## Transforming data using the `perm` function

The main challenge in implementing (3) in Python lies in de-meaning the variables, i.e., constructing $\ddot{y}_{it}=y_{it}-\bar{y}_{i}$
and $\mathbf{\ddot{x}}_{it}=\mathbf{x}_{it}-\mathbf{\bar{x}}_{i}$.
On the *individual level*, this can be done by premultiplying equation (1) by a transformation matrix

\begin{equation}
\mathbf{Q}_{T}:=\mathbf{I}_{T}-\left(\begin{array}{ccc}
1/T & \ldots & 1/T\\
\vdots & \ddots & \vdots\\
1/T & \ldots & 1/T
\end{array}\right)_{T\times T}.
\end{equation}

However, even though $\mathbf{Q}_{T}\mathbf{y}_{i}=\ddot{\mathbf{y}}_{i}$, we can't simply multiply the full data vector, $\mathbf{y}=(\mathbf{y}_{1},\dots,\mathbf{y}_{N})'$, with $\mathbf{Q}_{T}$ since it needs to be done for each individual. Towards this end, the Python function `perm(P,x)` picks out the elements of the input-vector (here `x`) and premultiplies
with the input-matrix `P` for one individual at the time (using
a `for` loop). For example, 
\begin{align*}
`perm`\left(\mathbf{Q}_{T},\begin{pmatrix}\mathbf{y}_{1}\\
\vdots\\
\mathbf{y}_{N}
\end{pmatrix}\right)=\begin{pmatrix}\mathbf{Q}_{T}\mathbf{y}_{1}\\
\vdots\\
\mathbf{Q}_{T}\mathbf{y}_{N}
\end{pmatrix}=\begin{pmatrix}\ddot{\mathbf{y}}_{1}\\
\vdots\\
\ddot{\mathbf{y}}_{N}
\end{pmatrix} & =\ddot{\mathbf{y}}.
\end{align*}

The same goes for $\textbf{x}$-input. (You may want to
take a look under the hood of this function.)

## Exercises with FE --- Within-Groups Estimation

The exercise takes up the union membership example from before. The data set WAGEPAN.TXT contains information about 545 men who worked every year from 1980 to 1987 in the US. The variables of interest are


| Variable | Content |
|-|-|
| nr | Variable that identifies the individual  |
| year | Year of observation |
| Black | Black |
| Hisp | Hispanic |
| Educ | Years of schooling |
| Exper | Years since left school |
| Expersq | Exper2 |
| Married | Marital status |
| Union | Union membership |
| Lwage | Natural logarithm of hourly wages |

Consider the following wage equation:

$$
\begin{align}
\ln\left(wage_{it}\right) & =\beta_{0}+\beta_{1}\textit{exper}_{it}+\beta_{2}\textit{exper}_{it}^{2}+\beta_{3}\textit{union}_{it}+\beta_{4}\textit{married}_{i}\nonumber \\
 & \quad+\beta_{5}\textit{educ}_{i}+\beta_{6}\textit{hisp}_{i}+\beta_{7}\textit{black}_{i}+c_{i}+u_{it} \tag{4}
\end{align}
$$

Note that *educ, *hisp*, and *black* are time-invariant variables.

# Exercises

Start by loading the data. Some of this has been done for you already. Since we are working with panels, we need to know how many persons there are and how many time periods we observe them. Since we operate using a balanced panel, this makes our life a little easier.

In [ ]:
# First, import the data into numpy. 
# Data should load the .txt file.
data =  # Fill in
id_array = np.array(data[:, 0])

# Count how many persons we have. This returns a tuple with the unique IDs,
# and the number of times each person is observed.
unique_id = np.unique(id_array, return_counts=True)
n = unique_id[0].size
t = int(unique_id[1].mean())
year = np.array(data[:, 1], dtype=int)

The table above does not correspond 1:1 with the text file. The data has 10 columns. Named from 0 to 9. Here is a variable describtion:
- Column 0: ID
- Column 1: Year
- Column 2: Black
- Column 3: Experience
- Column 4: Hispanic
- Column 5: Married
- Column 6: Education
- Column 7: Union
- Column 8: ln wage
- Column 9: Experience sqr

In [ ]:
# Load the rest of the data into arrays.
y =  # Fill in

# x needs to have a constant vector in the first row.
# How would you add this? x should have the shape of (n*t, 8).
x =  # Fill in

# Lets also make some variable names
label_y = 'Log wage'
label_x = [
    'Constant', 
    'Black', 
    'Hispanic', 
    'Education', 
    'Experience', 
    'Experience sqr', 
    'Married', 
    'Union'
]

## FE Questions
### FE (a):
- **Estimate (4) by pooled OLS,** thus considering for the moment the unobserved components of (4) as one (composite) error term $v_{it}=c_{i}+u_{it}$. 
- What assumptions are made about $E\left[c_{i}\mathbf{x}_{it}\right]$ and $E\left[u_{it}\mathbf{x}_{it}\right]$ when justifying this estimation approach?

In [ ]:
# Fill in

Use `print_table` and you should get a table that look like this:

Pooled OLS <br>
Dependent variable: Log wage <br>

|                |    Beta |     Se |   t-values |
|----------------|---------|--------|------------|
| Constant       | -0.0347 | 0.0646 |    -0.5375 |
| Black          | -0.1438 | 0.0236 |    -6.1055 |
| Hispanic       |  0.0157 | 0.0208 |     0.7543 |
| Education      |  0.0994 | 0.0047 |    21.2476 |
| Experience     |  0.0892 | 0.0101 |     8.8200 |
| Experience sqr | -0.0028 | 0.0007 |    -4.0272 |
| Married        |  0.1077 | 0.0157 |     6.8592 |
| Union          |  0.1801 | 0.0171 |    10.5179 |
R² = 0.187 <br>
σ² = 0.231

### FE (b):
- Within transform the data using the `perm` function. What happens to *educ, hisp, and black* and $x_{it1}\equiv1$ when the data are within transformed? 
- What is the rank of the within transformed $\mathbf{X}$ matrix? Why?

In [ ]:
# Fill in

### FE (c):
- Estimate (4) on within transformed data (make sure that the employed $\mathbf{\ddot{X}}$ has full rank - drop columns if necessary). 
- How big is the union premium according to the estimate from the FE model? Compare this with the estimate that you calculated from the pooled OLS regression. What does this suggest about $E\left[union_{it}c_{i}\right]$?

In [ ]:
# Fill in

You should get a table that look like this:

FE regression<br>
Dependent variable: Log wage

|                |    Beta |     Se |   t-values |
|----------------|---------|--------|------------|
| Experience     |  0.1168 | 0.0084 |    13.8778 |
| Experience sqr | -0.0043 | 0.0006 |    -7.1057 |
| Married        |  0.0453 | 0.0183 |     2.4743 |
| Union          |  0.0821 | 0.0193 |     4.2553 |
R² = 0.178 <br>
σ² = 0.123

## Exercises with first-difference estimation (FD)

The within transformation is one particular transformation
that enables us to get rid of $c_{i}$. An alternative is the first-difference transformation. To see how it works, lag Equation (1) one period and subtract it from (1) such that

\begin{equation}
\Delta y_{it}=\Delta\mathbf{x}_{it}'\mathbf{\beta}+\Delta u_{it},\quad t=\color{red}{2},\dotsc,T, \tag{5}
\end{equation}

where $\Delta y_{it}:=y_{it}-y_{it-1}$, $\Delta\mathbf{x}_{it}:=\mathbf{x}_{it}-\mathbf{x}_{it-1}$
and $\Delta u_{it}:=u_{it}-u_{it-1}$. As was the case for the within
transformation, first differencing eliminates the time invariant component
$c_{i}$. Note, however, that one time period is lost when differencing.

### FD Assumptions

\begin{eqnarray*}
FD.1 & : & E[u_{it}|\mathbf{x}_{i1},..,\mathbf{x}_{iT},c_{i}]=0 \; \; \; \; t=1,\dots,T, \; \;  \text{(as in }FE.1\text{)} \\
FD.2 & : & \text{Rank }E[\Delta\mathbf{X}_{i}\Delta\mathbf{X}_{i}']=K,\quad\text{ for }i=1,\dots,N,\\
FD.3 & : & E[\mathbf{e}_{i}\mathbf{e}_{i}'|\mathbf{x}_{i},c_{i}]=\sigma_{u}^{2}\mathbf{I}_{T-1} \; \text{ with }\mathbf{e}_{i}:=\Delta\mathbf{u}_{i},\quad\text{ for }i=1,\dots,N.
\end{eqnarray*}

Under the strict exogeneity assumption ($FD.1$) and the rank condition ($FD.2$), the FD estimator

\begin{equation}
\hat{\beta}_{FD}=\left(\Delta\mathbf{X}'\Delta\mathbf{X}\right)^{-1}\Delta\mathbf{X}^{\prime}\Delta\mathbf{y}
\end{equation}

consistently estimates $\boldsymbol{\beta}$ (as $N\to\infty$ for
fixed $T$). If also FD.3 holds, then $\hat{\boldsymbol{\beta}}_{FD}$
is asymptotically efficient. (Again, the latter assumption is not
needed for consistency.)

Under $FD.3$, $u_{it}=u_{it-1}+e_{it}$ follows a random walk. This
is the opposite extreme relative to assumption $FE.3$, where the
$u_{it}$ are assumed to be serially uncorrelated. In many cases,
the truth is likely to lie somewhere in between. The variance-covariance
matrix of the FE estimator may be estimated by

\begin{equation}
\widehat{\text{var}}(\hat{\mathbf{\beta}}_{FD})=\hat{\sigma}_{e}^{2}\left(\Delta\mathbf{X}'\Delta\mathbf{X}\right)^{-1}
\end{equation}

where $\hat{\sigma}_{e}^{2}:=\hat{\mathbf{e}}^{\prime}\hat{\mathbf{e}}/[N\left(T-1\right)-K]$
and $\hat{e}_{it}:=$ $\Delta y_{it}-\Delta\mathbf{x}_{it}\widehat{\mathbf{\beta}}$.

Notice how we, both in the case of FE and FD, manipulate the model
in a way that allows the standard OLS assumptions to hold on the *transformed* data, and then simply treat the transformed model as if it was our model of interest. Under exogeneity ($FE.1/FD.1$) the choice between first difference and the within estimator pertains to efficiency considerations, and the choice hinges on the assumptions made about the serial correlation of the errors $(FE.3$$/$$FD.3)$.

To estimate the coefficients in (5) in Python, we must first
difference all the variables, i.e construct $\Delta y_{it}=y_{it}-y_{it-1}$ and $\Delta\mathbf{x}_{it}=\mathbf{x}_{it}-\mathbf{x}_{it-1}$. This can be done by premultiplying the variables in levels ($y_{i}$ and $\mathbf{x}_{i})$ by the transformation matrix $\mathbf{D}$ given by

\begin{equation}
\mathbf{D}:=\left(\begin{array}{cccccc}
-1 & 1 & 0 & \ldots & 0 & 0\\
0 & -1 & 1 &  & 0 & 0\\
\vdots &  &  & \ddots &  & \vdots\\
0 & 0 & 0 & \ldots & -1 & 1
\end{array}\right)_{T - 1\times T}.
\end{equation}

(Can you see why $\mathbf{D}$ gets the job done?)

## FD Questions
### FD (a):
- Construct $\mathbf{D}$ and use the procedure `perm` $(\mathbf{D},\mathbf{x})$ to compute first differences of the elements of $\mathbf{y}$ and $\mathbf{x}$. 
- What happens to *educ, hisp* and *black* and $x_{it1}\equiv1$ when the data are transformed into first differences? What is the rank of the first differenced $\mathbf{x}$-matrix? Why?

In [ ]:
# Fill in

### FD (b):
- **Estimate (4) in first differences.** How big is the union premium according to the estimate from this model? 
- Compare the FD estimate with the estimate that you calculated from the FE regression. Is there a difference? If yes, what (if anything) can we conclude based on this finding?

In [ ]:
# Fill in

You should get a table that look like this:

FD regression <br>
Dependent variable: Log wage

|                |    Beta |     Se |   t-values |
|----------------|---------|--------|------------|
| Experience     |  0.1158 | 0.0196 |     5.9096 |
| Experience sqr | -0.0039 | 0.0014 |    -2.8005 |
| Married        |  0.0381 | 0.0229 |     1.6633 |
| Union          |  0.0428 | 0.0197 |     2.1767 |
R² = 0.004 <br>
σ² = 0.196

## Exercise comparing FE and FD
### Question FE v. FD (a):
**Test for serial correlation in the errors using an auxilliary AR(1) model**, to test assumption FD.3, where the errors $e_{it} = \Delta u_{it}$ should be serially uncorrelated.

We can easily test this assumption given the OLS residuals from equation (5). Run the regression (note that you will loose data for
the *two* first periods)
\begin{equation}
\hat{e}_{it}=\rho\hat{e}_{it-1}+error_{it},\quad t=\color{red}{3},\dotsc,T,\quad i=1,\dotsc,N
\end{equation}

Do you find any evidence for serial correlation? Does FD.3 seem appropriate? And why don't we include an intercept in this auxilliary equation?

*Note:* Under FE.3, the idiosyncratic errors $u_{it}$
are uncorrelated. However, FE.3 implies that the $e_{it}$'s are autocorrelated. In fact, of the $u_{it}$'s are serially uncorrelated to beging with, corr$\left(e_{it},e_{it-1}\right)=-0.5$. (Check!) This test is of course only valid if the explanatory variables are strictly exogenous!

*Hint:* You can use the `perm` function to lag
the error term variable. Consider the following; 
\begin{align*}
\underset{T-1\times T}{\begin{pmatrix}
1 & 0 & 0 & \cdots & 0 & 0\\
0 & 1 & 0 & \cdots & 0 & 0\\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots\\
0 & 0 & 0 & \cdots & 1 & 0
\end{pmatrix}}\underset{T \times 1}{\begin{pmatrix}y_{1}\\
y_{2}\\
\vdots\\
y_{T}
\end{pmatrix}}=\underset{T - 1\times 1}{\begin{pmatrix}y_{1}\\
y_{2}\\
\vdots\\
y_{T - 1}
\end{pmatrix}}
\end{align*}

In [ ]:
def serial_corr(y, x, year):
    # Fill in
    # e_l are the lagged values of e.
    return lm.estimate(e, e_l)

In [ ]:
corr_result = serial_corr(y_diff, x_diff, year)

label_ye = 'OLS residual, e\u1d62\u209c'
label_e = ['e\u1d62\u209c\u208B\u2081']
title = 'Serial Correlation'
lm.print_table(
    (label_ye, label_e), corr_result, 
    title='Serial Correlation', floatfmt='.4f'
)

*Hint:* Remember again to remove the first year after you have lagged the residuals. So in your estimations your residuals should be excluded the years 1980 and 1981.

You should get a table that look like this:

Serial Correlation <br>
Dependent variable: OLS residual, eᵢₜ

|       |    Beta |     Se |   t-values |
|-------|---------|--------|------------|
| eᵢₜ₋₁ | -0.3961 | 0.0147 |   -27.0185 |
R² = 0.182 <br>
σ² = 0.143

### Question FE v FD (b):

Test for strict exogeneity: Add a lead of the union variable, $unioni_{t+1}$ to the equation (4) (note that you will lose data from period $T$ , 1987) and estimate the model with *fixed effects* (i.e., you have to demean $unioni_{t+1}$ along with all the other variables and throw out time constant variables). Is $unioni_{t+1}$ significant? What does this imply for the strict exogeneity assumption?

*Hint:* To lead a variable, think along the same lines as in Question FE v FD (a)

In [1]:
# Fill in

The table should look something like this:
Exogeneity test <br>
Dependent variable: Log wage

|                |    Beta |     Se |   t-values |
|----------------|---------|--------|------------|
| Experience     |  0.1213 | 0.0100 |    12.1001 |
| Experience sqr | -0.0050 | 0.0008 |    -6.3579 |
| Married        |  0.0436 | 0.0209 |     2.0898 |
| Union          |  0.0757 | 0.0218 |     3.4784 |
| Union lead     |  0.0515 | 0.0223 |     2.3063 |
R² = 0.146<br>
σ² = 0.128

### Question FE v FD (b):
Add interactions on the form $d_{81}\cdot educ, d_{82}\cdot educ, ..., d_{87}\cdot educ$ and estimate the model with fixed effect. Has the return to education increased over time?

*Hint:* Remember that $educ_{i}$ doesn't vary over
time! Therefore we didn't use $educ$ in levels in the FE estimation.
However, if we suppose that the structural equation (4) contains a term $\sum_{s=2}^{T}\delta_{s}d_{s}educ_{i}$, it will be perfectly fine to within-transform these interactions since they vary over time (although in a highly structured manner - they equal
zero in all time periods but one, and then $educ$). Note that one
period is dropped for the within-transformation to work whereas the
levels term, $\beta_{5}educ_{i}$, is dropped to avoid producing a
constant row.

*Programming hint:* You want to append the dataset with a dummy matrix, that would look something like this:

$$
\begin{bmatrix}
0 & 0 & 0 & 0 & 0 & 0 & 0 \\
14 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 14 & 0 & 0 & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & 0 & 0 & 0 & 14 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 \\
9 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 9 & 0 & 0 & 0 & 0 & 0 \\
\vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & 0 & 0 & 0 & 9 \\
\end{bmatrix}
$$

This example shows our two first persons, that have 14 and 9 years of education respectively. This matrix can be created as a product of two matrices, what would they look like (a dummy matrix with onlyn ones can be creates using `np.eye` and `np.tile`)? Why is the first row for each person only zeros?

In [ ]:

# Fill in

label_x_interactions = label_x_fe + [
    'E81', 'E82', 'E83', 'E84', 'E85', 'E86', 'E87'
]

You should get a table that look like this:

FE with year interactions <br>
Dependent variable: Log wage

|                |    Beta |     Se |   t-values |
|----------------|---------|--------|------------|
| Experience     |  0.1705 | 0.0273 |     6.2462 |
| Experience sqr | -0.0060 | 0.0009 |    -6.9581 |
| Married        |  0.0475 | 0.0183 |     2.5925 |
| Union          |  0.0794 | 0.0193 |     4.1138 |
| E81            | -0.0010 | 0.0026 |    -0.4009 |
| E82            | -0.0062 | 0.0041 |    -1.5224 |
| E83            | -0.0114 | 0.0057 |    -2.0006 |
| E84            | -0.0136 | 0.0072 |    -1.8787 |
| E85            | -0.0162 | 0.0087 |    -1.8578 |
| E86            | -0.0170 | 0.0101 |    -1.6804 |
| E87            | -0.0167 | 0.0115 |    -1.4619 |
R² = 0.181 <br>
σ² = 0.123